In [1]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta


Conexión a Base de Datos y formateo de fechas "Desde" y "Hasta" para bajar la data de CAMMESA

In [2]:
# Lectura de archivo excel (Reemplazado por base de datos SQL en el futuro)
archivo_excel = r'df_POSOP.xlsx'

df = pd.read_excel(archivo_excel)

# Converte la columna FECHA al formato datetime si no está en ese formato
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Encuentra la última fecha y la convertierte al formato YYMMDD para servir como referencia al próximo archivo a descargar
ultima_fecha = df['FECHA'].max()
#Le suma un día para ir a buscar a CAMMESA
Fecha_desde= ultima_fecha + timedelta(days=1)

#Formateo Fecha para consutla de CAMMESA y para armado del nombre para extraer zip
Fecha_desde_n = Fecha_desde.strftime('%y%m%d')
Fecha_desde = Fecha_desde.strftime('%Y-%m-%dT%H:%M:%S.000-03:00')


# Obtiene la fecha actual
Fecha_hasta = datetime.now()

#Formateo Fecha para consutla de CAMMESA y para armado del nombre para extraer zip
Fecha_hasta_n = Fecha_hasta.strftime('%y%m%d')
Fecha_hasta = Fecha_hasta.strftime('%Y-%m-%dT%H:%M:%S.000-03:00')


In [4]:
Fecha_hasta

'2024-02-07T16:37:41.000-03:00'

Consulta de Info a CAMMESA

In [2]:
#URL para capturar Id del documento y el zip file:

#Defino la tabla de CAMMESA que me voy a traer
nemo = "PARTE_POST_OPERATIVO"
#Busco los zip disponibles para traer así puedo extraer el id
URL = f"https://api.cammesa.com/pub-svc/public/findDocumentosByNemoRango?fechadesde={Fecha_desde}&fechahasta={Fecha_hasta}&nemo={nemo}"
response = requests.get(URL)
PPO=response.json()

NameError: name 'Fecha_desde' is not defined

In [36]:
base_zip_file_url = "https://api.cammesa.com/pub-svc/public/findAllAttachmentZipByNemoId?docId={}&nemo=PARTE_POST_OPERATIVO"

for i in range(len(PPO)):
    # Construir la URL con el ID actual
    zip_file_url = base_zip_file_url.format(PPO[i]['id'])
    
    # Realizar la solicitud GET para obtener el contenido del archivo ZIP
    r = requests.get(zip_file_url)
    
    # Crear un objeto ZipFile a partir del contenido descargado
    z = zipfile.ZipFile(io.BytesIO(r.content))
    
    # Directorio de destino para extraer los archivos ZIP
    destination_directory = "C:\\Users\\cvillena\\Downloads"
    
    # Extraer todos los archivos del ZIP en el directorio específico
    z.extractall(destination_directory)

Extracción de datos a un dataframe

In [68]:
#Genero función que me de los nombres de los zip en el formato deseado

def generar_nombres_archivos(desde, hasta):
    nombres_archivos = []
    for year in range(int(desde[:2]), int(hasta[:2]) + 1):
        for month in range(int(desde[2:4]), int(hasta[2:4]) + 1):
            for day in range(int(desde[4:]), int(hasta[4:]) + 1):
                nombre_archivo = f"PO{year:02d}{month:02d}{day:02d}.zip"
                nombres_archivos.append(nombre_archivo)
    return nombres_archivos



# Generar los nombres de los archivos ZIP mensuales dentro del rango especificado
archivos_zip = generar_nombres_archivos(Fecha_desde_n, Fecha_hasta_n)


In [70]:
dfs=[]
for archivo_zip in archivos_zip:
    try:
        # Ruta completa del archivo ZIP mensual
        ruta_completa = os.path.join(destination_directory, archivo_zip)
        print(archivo_zip)
        
        # Extrae el archivo MDB de cada archivo ZIP mensual
        with zipfile.ZipFile(ruta_completa, 'r') as zip_ref:
            # Encontrar el nombre del archivo MDB dentro del ZIP mensual
            archivo_mdb = os.path.splitext(archivo_zip)[0] + ".mdb"
            zip_ref.extract(archivo_mdb, path=destination_directory)

        # Lee el archivo MDB y cargar la tabla VALORES_GENERADORES en un dataframe
        mdb_file = os.path.join(destination_directory, archivo_mdb)
        conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={mdb_file};"
        conn = pyodbc.connect(conn_str)
        df = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)
        conn.close()

        # Agregar el dataframe a la lista
        dfs.append(df)
    except FileNotFoundError:
        print(f"El archivo {archivo_zip} no se encontró. Saltando al siguiente archivo...")
        continue

# Concatenar verticalmente todos los dataframes mensuales en uno solo
df_final = pd.concat(dfs, ignore_index=True)



PO240204.zip
PO240205.zip
PO240206.zip
El archivo PO240206.zip no se encontró. Saltando al siguiente archivo...
PO240207.zip
El archivo PO240207.zip no se encontró. Saltando al siguiente archivo...


Subida de dataframe a base de datos

In [71]:
# Por limitaciones de tamaño de excel filtramos solo las máquinas Pampa
valores_filtrados = ["ADTOHI", "AR21EO", "BAHIEO", "BBLATV29", "BBLATV30",
                      "BBLMDI01", "BBLMDI02", "BBLMDI03", "BBLMDI04", 
                      "BBLMDI05", "BBLMDI06", "CERITV01", "CORTEO", 
                      "EBARTG01", "EBARTG02", "EBARTV01", "ETIGHI", 
                      "GEBATG01", "GEBATG02", "GEBATG03", "GEBATG04", 
                      "GEBATV01", "GEBATV02", "GUEMTG01", "GUEMTV11", 
                      "GUEMTV12", "GUEMTV13", "LDLATG01", "LDLATG02", 
                      "LDLATG03", "LDLATG04", "LDLATG05", "LDLATV01", 
                      "LDLMDI01", "LREYHB", "NIH1HI", "NIH2HI", "NIH3HI", 
                      "PAMEEO", "PEP3EO", "PILBDI01", "PILBDI02", 
                      "PILBDI03", "PILBDI04", "PILBDI05", "PILBDI06", "PIQIDI01", "PPLEHI"]

# Filtrar el DataFrame por los valores especificados en la columna "GRUPO"
df_filtrado = df_final[df_final['GRUPO'].isin(valores_filtrados)]


In [74]:
df_concatenado = pd.concat([df_final, df], ignore_index=True)



In [19]:
# Escribir el DataFrame final en un archivo Excel
ruta_excel = "C:\Users\jadurian\OneDrive - Pampa Energia\Laburo-Estudio\Proyectos\Tablero de Análisis de Pérdidas de Ingresos\VSCode\Data\df_POSOP.xlsx"
df_concatenado.to_excel(ruta_excel, index=False)
print("DataFrame exportado a Excel con éxito:", ruta_excel)


DataFrame exportado a Excel con éxito: C:\Users\cvillena\Downloads\df_POSOP.xlsx


In [ ]:
# Configuración de la conexión a SQL Server
path_zip = ".zips/PO240115.zip"
server = 'tu_servidor'
database = 'tu_base_de_datos'
username = 'tu_usuario'
password = 'tu_contraseña'
driver = '{ODBC Driver 17 for SQL Server}'

# Función para conectar y leer la tabla específica del archivo .mdb
def leer_tabla_mdb(nombre_tabla):
    # Conectar al archivo .mdb dentro del .zip
    with zipfile.ZipFile('archivo.zip') as zip_ref:
        with zip_ref.open('archivo.mdb') as mdb_file:
            conn_str = (
                r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
                r'DBQ=path_to_your_mdb_file'
            )
            
            conn = pyodbc.connect(conn_str)
            cursor = conn.cursor()
            
            # Leer datos de la tabla especificada
            cursor.execute(f'SELECT * FROM {nombre_tabla}')
            
            # Obtener todos los datos de la tabla
            datos_tabla = cursor.fetchall()
            
            conn.close()
            
            return datos_tabla
            pass

# Función para insertar los datos en SQL Server
def insertar_en_sql_server(datos, nombre_tabla_sql):
    # Código para conectar a la base de datos SQL Server con pyodbc
    conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')
    cursor = conn.cursor()

    # Crear una tabla si no existe
    cursor.execute(f"IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'{nombre_tabla_sql}') CREATE TABLE {nombre_tabla_sql} (columna1 INT, columna2 VARCHAR(255))")

    # Insertar los datos en la tabla de SQL Server
    for dato in datos:
        cursor.execute(f"INSERT INTO {nombre_tabla_sql} VALUES (?, ?)", dato)

    conn.commit()
    conn.close()

# Lógica principal
nombre_tabla_mdb = 'nombre_tabla_mdb'
nombre_tabla_sql = 'nombre_tabla_sql'

datos_tabla_mdb = leer_tabla_mdb(nombre_tabla_mdb)

if datos_tabla_mdb:
    insertar_en_sql_server(datos_tabla_mdb, nombre_tabla_sql)